# 1) Kod: Genel Shift Modülü (PyTorch)

Bu sürüm:

* İstediğin kadar yön (G adet) alır

* Kanal sayısını otomatik gruplara böler

* torch.roll ile kaydırır

* Wrap-around’ı (dönüp dolaşıp diğer taraftan gelmesini) sıfırlayarak iptal eder

In [ ]:
import torch
import torch.nn as nn

class Shift2D(nn.Module):
    """
    Genel Shift Operatörü
    - Kanalları G gruba böler
    - Her grubu (dx, dy) ile kaydırır
    - Sınır taşmalarını 0'lar (zero padding etkisi)
    """
    def __init__(self, directions=None):
        super().__init__()
        if directions is None:
            # center + 4 yön
            directions = [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]
        self.directions = directions

    @staticmethod
    def _zero_out_wrap(rolled, dx, dy):
        # rolled: (B, Cg, H, W)
        if dy > 0:
            rolled[..., :dy, :] = 0
        elif dy < 0:
            rolled[..., dy:, :] = 0

        if dx > 0:
            rolled[..., :, :dx] = 0
        elif dx < 0:
            rolled[..., :, dx:] = 0

        return rolled

    def forward(self, x):
        B, C, H, W = x.shape
        G = len(self.directions)

        base = C // G
        sizes = [base] * (G - 1) + [C - base * (G - 1)]
        chunks = torch.split(x, sizes, dim=1)

        out = []
        for chunk, (dx, dy) in zip(chunks, self.directions):
            if dx == 0 and dy == 0:
                out.append(chunk)
                continue

            rolled = torch.roll(chunk, shifts=(dy, dx), dims=(-2, -1))
            rolled = self._zero_out_wrap(rolled, dx, dy)
            out.append(rolled)

        return torch.cat(out, dim=1)

# Hızlı test
if __name__ == "__main__":
    x = torch.randn(2, 10, 8, 8)  # B=2, C=10
    shift = Shift2D()
    y = shift(x)
    print("x:", x.shape, "y:", y.shape)


x: torch.Size([2, 10, 8, 8]) y: torch.Size([2, 10, 8, 8])


# 2) Bu “genel shift” tanımında kritik kararlar

### Yön seti (directions)
Varsayılan 5 yön iyi bir başlangıç:

* (0,0), (1,0), (-1,0), (0,1), (0,-1)

### Kanal dağıtımı
Burada C // G bölüp son gruba kalanı veriyoruz.
* İstersek daha kontrollü dağıtım da yapılır.

### Padding davranışı
* Biz “zero padding etkisi” yaptık.
* Bu, literatürdeki en yaygın basit yaklaşımdır.

----
-----


# Shift2D Kodunda Genel Olarak Ne Yapılıyor? 

Bu kodun amacı: **(B, C, H, W)** boyutundaki bir feature map’te, kanalları gruplara ayırıp her grubu farklı bir yöne kaydırarak (shift) **parametresiz bir “uzamsal karıştırma (spatial mixing)”** yapmak.



## 1) Girdi ve Hedef

- Girdi: `x` tensoru  
  **Şekil:** `(B, C, H, W)`
  - `B`: batch size
  - `C`: kanal sayısı
  - `H, W`: yükseklik, genişlik

- Çıktı: `y` tensoru  
  **Şekil yine:** `(B, C, H, W)`

Ama içerik değişiyor: bazı kanalların pikselleri **sağa/sola/yukarı/aşağı kaymış** oluyor.

## 2) `directions` Nedir?

```python
directions = [(0,0), (1,0), (-1,0), (0,1), (0,-1)]
```
Her eleman bir yön gösterir:

(dx, dy):

* dx = +1 → sağa kaydır

* dx = -1 → sola kaydır

* dy = +1 → aşağı kaydır

* dy = -1 → yukarı kaydır

* (0,0) → kaydırma yok (kanal olduğu gibi kalır)

Bu liste kaç eleman ise, kanallar o kadar gruba ayrılır.


## 3) Kanalları Gruplara Ayırma Mantığı

Kod:
```python
G = len(self.directions)
base = C // G
sizes = [base] * (G - 1) + [C - base * (G - 1)]
chunks = torch.split(x, sizes, dim=1)
```


* G: yön sayısı (örneğin 5)

* C: kanal sayısı (örneğin 32)

* base = C // G: her gruba yaklaşık kaç kanal düşeceği

Sonra torch.split ile x kanal boyutundan parçalanır:

* chunks artık bir liste:

* chunks[0] → ilk kanal grubu (örneğin 6 kanal) → direction (0,0)

* chunks[1] → ikinci grup → direction (1,0)

* ...

Yani her grup farklı yöne kaydırılacak.

## 4) Her Grubu Kaydırma (torch.roll)

Kod:
```python
rolled = torch.roll(chunk, shifts=(dy, dx), dims=(-2, -1))
```


* torch.roll tensoru kaydırır.

dims=(-2, -1) demek:

* -2 → H ekseni

* -1 → W ekseni

Yani:

* dy kadar yukarı/aşağı,

* dx kadar sola/sağa kaydırır.

>Ama önemli: torch.roll “wrap-around” yapar:

* sağa kaydırınca sağdan taşan değerler soldan geri girer.
* Bu gerçek bir padding davranışı değildir, sahte bilgi üretir.

## 5) Wrap-around’ı Sıfırlama (Zero Padding Etkisi)

Kod:
```python
rolled[..., :dy, :] = 0
rolled[..., dy:, :] = 0
rolled[..., :, :dx] = 0
rolled[..., :, dx:] = 0
```


* Bu kısım, roll sonrası oluşan “diğer taraftan geri gelen” bölgeleri 0 yapar.

Böylece şuna benzer bir etki oluşur:

* “Kaydırma yaptım, dışarı taşanı attım, boş kalan kısmı 0 ile doldurdum.”

* Yani zero padding gibi davranmış oluruz.

## 6) Grupları Yeniden Birleştirme

Kod:
```python
return torch.cat(out, dim=1)
```


* Her yön için kaydırılmış kanal grupları out listesinde birikiyor.
* Sonra torch.cat(..., dim=1) ile tekrar kanal ekseninde birleştirilip:

* Çıkış y elde ediliyor: (B, C, H, W)